In [1]:
import torch
import numpy as np
import gym
gym.logger.set_level(40)

from ppo import PPO
from test_policy import test_policy, plot_animation
from network import FeedForwardNN

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
env = gym.make('LunarLanderContinuous-v2')

obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
print('obs_dim, act_dim', obs_dim, act_dim)


actor = FeedForwardNN(obs_dim, act_dim)
critic = FeedForwardNN(obs_dim, 1)

savename = 'lunarlandercont'
actor.load_state_dict(torch.load('../Models/'+savename+'ppo_actor.pth'))
critic.load_state_dict(torch.load('../Models/'+savename+'ppo_critic.pth'))

model = PPO(env, actor, critic)

obs_dim, act_dim 8 2


# Solved is 200 points

https://xaviergeerinck.com/post/ai/rl/openai-lunar-lander

The landing pad is always at coordinates (0,0). The coordinates are the first two numbers in the state vector.
Reward for moving from the top of the screen to the landing pad and zero speed is about 100..140 points.
If the lander moves away from the landing pad it loses reward. The episode finishes if the lander crashes or
comes to rest, receiving an additional -100 or +100 points. Each leg with ground contact is +10 points.
Firing the main engine is -0.3 points each frame. Firing the side engine is -0.03 points each frame.
Solved is 200 points.
Landing outside the landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land
on its first attempt. Please see the source code for details.

stats space is dim 8

1,2: first 2 are position in x axis and y axis(hieght) 
3,4: other 2 are the x,y axis velocity terms, 
5,6: lander angle and angular velocity, 
7,8: left and right left contact points

Observation Space: The observation space is illustrated by a "Box" containing 8 values between [ $-\infty$, $\infty$ ] these values are:

- Position X
- Position Y
- Velocity X
- Velocity Y
- Angle
- Angular Velocity
- Is left leg touching the ground: 0 OR 1
- Is right leg touching the ground: 0 OR 1

Action Space:

Discrete (Discrete Action Space with 4 values):
- 0 = Do Nothing
- 1 = Fire Left Engine
- 2 = Fire Main Engine
- 3 = Fire Right Engine

Continuous (Box Action Space with 2 values between -1 and +1):
- Value 1: [-1.0, +1.0] for main engine where [-1.0, 0.0] = Off and [0.0, +1.0] = On
- Value 2:
    -[-1.0, -0.5]: Left Engine
    - [-0.5, 0.5]: Off
    - [0.5, 1.0]: Right Engine

Reward Function:

The Reward Function is a bit more complex and consists out of multiple components:

- [100, 140] points for Moving to the landing pad and zero speed
- Negative reward for moving away from the landing pad
- If lander crashes or comes to rest it gets -100 or +100
- Each leg with ground contact gets +10
- Firing the main engine is -0.3 per frame
- Firing the side engine is -0.03 per frame
- Solved is 200 points

In [ ]:
model.learn(total_timesteps=1_000_000, track_progress = True, savename = 'lunarlandercont2')

In [10]:
# Build our policy the same way we build our actor model in PPO
env = gym.make('LunarLanderContinuous-v2')

env.seed(42) # 247 289.8606570638711

#torch.manual_seed(0) 
#policy = model.actor

# Load in the actor model saved by the PPO algorithm
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
actor = FeedForwardNN(obs_dim, act_dim)

savename = 'lunarlandercont2'
actor.load_state_dict(torch.load('Models/'+savename+'ppo_actor.pth'))
critic.load_state_dict(torch.load('Models/'+savename+'ppo_critic.pth'))
    
episode_len, episode_return, frames = test_policy(actor, env, render = True)
print('1 episode: total timesteps', episode_len, 'total rewards', episode_return) 

1 episode: total timesteps 293 total rewards 289.16941026360473


In [ ]:
plot_animation(frames)